<a href="https://colab.research.google.com/github/kesavakrishna1/3-2Project/blob/main/4.1/Copy_of_b_to_b_SCG_to_ECG_signal_1_without_bandpass_filter_DEC21_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.14.0


In [ ]:
from tensorflow.keras.layers import LayerNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-najliuwg
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-najliuwg
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyedflib

In [ ]:
!pip install padasip

In [ ]:
fileNames = []
fileName_str = []
#Change the number here to read different files
for i in range(10,11,1):
  name_str = ''
  if(i<10):
    name_str = 'b00'
  else:
    name_str = 'b0'
  fileNames.append(name_str+ str(i)+'.edf')
  fileName_str.append(name_str+ str(i))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

In [ ]:
import pyedflib
import numpy as np
from scipy import signal
from scipy.signal import butter, filtfilt
from sklearn.decomposition import FastICA
from sklearn.preprocessing import scale
import padasip as pa
from scipy import signal
from keras.layers import LeakyReLU
original_scg = []
original_ecg = []
class DataUtils:

    def __init__(self) -> None:
        super().__init__()
        self.fileNames = fileNames


    # def readData(self, sigNum, path="/content/drive/MyDrive/Colab Notebooks/basal/"):
    def readData(self, sigNum, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        file_name = path + self.fileNames[sigNum]
        #print(file_name)
        #file_name =  self.fileNames[sigNum]
        f = pyedflib.EdfReader(file_name)
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        print("Reading file:: ",file_name)
        print("different columns:: ",signal_labels)
        print("total number of samples",f.getNSamples())
        abdECG = np.zeros((1, f.getNSamples()[0]))
        scg = np.zeros((1, f.getNSamples()[0]))
        scg[0, :] = f.readSignal(3)
        scg = scale(scg, axis=1)
        #scg[0, :] = scale(self.butter_bandpass_filter(scg, 1, 200, 1000), axis=1)
        #for i in np.arange(0, n-2):
        abdECG[0, :] = f.readSignal(0)
        abdECG = scale(abdECG, axis=1)
        #abdECG = scale(self.butter_bandpass_filter(abdECG, 1, 110, 1000), axis=1)
        print("before downsampling",abdECG.shape[1])
        abdECG = signal.resample(abdECG, int(abdECG.shape[1] / 5), axis=1)
        scg = signal.resample(scg, int(scg.shape[1] / 5), axis=1)

        # # differencing code starts
        # scg = np.diff(scg)
        # abdECG = np.diff(abdECG)
        # # differencing code ends

        print("after downsampling",abdECG.shape[1])
        '''function for ecg identification, using pan-tompkins algorithm'''
        fs = []
        fs.append(f.getSampleFrequency(0))

        #changed the return sequence, so that converion will be from SCG to ECG
        return  scg, abdECG, fs

    def windowingSig(self, sig1, sig2, windowSize=15):
        signalLen = sig2.shape[1]
        signalsWindow1 = [sig1[:, int(i):int(i + windowSize)].transpose() for i in range(0, signalLen - windowSize, windowSize)]
        signalsWindow2 = [sig2[:, int(i):int(i + windowSize)].transpose() for i in range(0, signalLen - windowSize, windowSize)]
        print("SCG shape after windowing:: ",np.array(signalsWindow1).shape)
        print("ECG shape after windowing:: ",np.array(signalsWindow2).shape)

        return signalsWindow1, signalsWindow2

    def adaptFilterOnSig(self, src, ref):
        f = pa.filters.FilterNLMS(n=4, mu=0.1, w="random")
        for index, sig in enumerate(src):
            try:
                y, e, w = f.run(ref[index][:, 0], sig)
                ref[index][:, 0] = e
            except:
                pass

        return ref

    def calculateICA(self, sdSig, component=7):
        ica = FastICA(n_components=component, max_iter=1000)
        icaRes = []
        for index, sig in enumerate(sdSig):
            try:
                icaSignal = np.array(ica.fit_transform(sig))
                icaSignal = np.append(icaSignal, sig[:, range(2, 4)], axis=1)
                icaRes.append(icaSignal)
            except:
                pass
        return np.array(icaRes)

    def createDelayRepetition(self, signal, numberDelay=4, delay=10):
        signal = np.repeat(signal, numberDelay, axis=0)
        #print("createDelayRepetition::",signal.shape)
        for row in range(1, signal.shape[0]):
            signal[row, :] = np.roll(signal[row, :], shift=delay * row)

        #print("createDelayRepetition 2::",signal.shape)
        return signal

    def __butter_bandpass(self, lowcut, highcut, fs, order=5):
        #print("00")
        nyq = 0.5 * fs
        #print("0")
        low = lowcut / nyq
        #print("11")
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        #print("1")
        return b, a

    def butter_bandpass_filter(self, data, lowcut, highcut, fs, order=3, axis=1):
        #print("In butter_bandpass_filter")
        b, a = self.__butter_bandpass(lowcut, highcut, fs, order=order)
        y = filtfilt(b, a, data, axis=axis)
        #print("2")
        return y

In [ ]:
'''function for ecg identification, using pan-tompkins algorithm'''

def codeFn(ecg, fs):
    nyquist = 0.5 * fs
    low_cutoff = 5
    high_cutoff = 15
    b, a = butter(1, [low_cutoff/nyquist, high_cutoff/nyquist], btype='band')
    ecg_filt = filtfilt(b, a, ecg)

    b = np.array([1, 0, -1])
    ecg_diff = np.convolve(ecg_filt, b, mode='same')
    ecg_sq = ecg_diff ** 2

    ma_len = int(0.08 * fs)
    ecg_ma = np.convolve(ecg_sq, np.ones(ma_len)/ma_len, mode='same')

    qrs_idx, _ = find_peaks(ecg_ma, distance=int(0.2 * fs), height=0.2 * np.max(ecg_ma))

    return qrs_idx


In [ ]:
from sklearn.model_selection import train_test_split

#from Utils.DataUtils import DataUtils
import numpy as np
import copy

class TrainUtils:
    def __init__(self) -> None:
        super().__init__()
        self.dataUtils = DataUtils()

    def prepareData(self, delay=5, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/b010.edf"):
        scgAll, ecg, fs = self.dataUtils.readData(0,path)
        print(scgAll.shape)
        #print("scg shape:: ",ecg.shape)
        scgAll = scgAll[range(1), :]
        #print("Number of samples:: ",scg.shape)
        delayNum = scgAll.shape[0]
        ecgAll = self.dataUtils.createDelayRepetition(ecg, delayNum, delay)
        #print("scg all shape:: ",ecgAll.shape)
        for i in range(1, len(fileNames)):
            scg, ecg = self.dataUtils.readData(i,path)
            print("Number of samples:: ",ecg.shape)
            scg = scg[range(1), :]
            ecgDelayed = self.dataUtils.createDelayRepetition(ecg, 1, delay)
            scgAll = np.append(scgAll, scg, axis=1)
            ecgAll = np.append(ecgAll, ecgDelayed, axis=1)
        #print("SCG all merged shape:: ", scg.shape)
        print("ECG all merged shape:: ", ecgAll.shape)

        original_scg = scgAll
        original_ecg = ecgAll
        scgWindows, ecgWindows = self.dataUtils.windowingSig(scgAll, ecgAll, windowSize=1000)
        # ecgWindows = self.dataUtils.adaptFilterOnSig(scgWindows, ecgWindows)
        # scgWindows = self.dataUtils.calculateICA(scgWindows, component=2)
        return scgWindows, ecgWindows

    def trainTestSplit(self, sig, label, trainPercent, shuffle=True):
        print("Splitting into train and test:: ")
        X_train, X_test, y_train, y_test = train_test_split(sig, label, train_size=trainPercent, shuffle=False)
        X_train = np.array(X_train)
        #X_train = self.window_the_data(X_train,  60)
        X_test = np.array(X_test)
        #X_test = self.window_the_data(X_test,  60)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        return X_train, X_test, y_train, y_test

    def window_the_data(self,data,window=32):
      r = list()
      for i, mat in enumerate(data):
        s = 0
        a = list()
        print(data.shape)
        while s + window < 256:
          a.append(copy.deepcopy(mat[s : s + window].T))
          # for i,x in enumerate(a):
            # a[i] = scaler.fit_transform(x)
          s += 2
        r.append(copy.deepcopy(a))
        #print(r.shape())


      r = np.array(r)
      return copy.deepcopy(r)

In [ ]:
# '''function for ecg identification, using pan-tompkins algorithm'''

# def codeFn(ecg, fs):
#     nyquist = 0.5 * fs
#     low_cutoff = 5
#     high_cutoff = 15
#     b, a = butter(1, [low_cutoff/nyquist, high_cutoff/nyquist], btype='band')
#     ecg_filt = filtfilt(b, a, ecg)

#     b = np.array([1, 0, -1])
#     ecg_diff = np.convolve(ecg_filt, b, mode='same')
#     ecg_sq = ecg_diff ** 2

#     ma_len = int(0.08 * fs)
#     ecg_ma = np.convolve(ecg_sq, np.ones(ma_len)/ma_len, mode='same')

#     qrs_idx, _ = find_peaks(ecg_ma, distance=int(0.2 * fs), height=0.2 * np.max(ecg_ma))

#     return qrs_idx

# # f=readData(,,)
# fs = f.getSampleFrequency(0)

In [ ]:
!pip install --upgrade keras

  Using cached keras-3.0.1-py3-none-any.whl (999 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0.post1 requires keras<2.16,>=2.15.0, but you have keras 3.0.1 which is incompatible.


In [ ]:
!pip install --upgrade tensorflow


  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.0.1
    Uninstalling keras-3.0.1:
      Successfully uninstalled keras-3.0.1


    **added 2 discriminators**

---

* line 103, 104 self.image shape c and d
* line 124 - 140 discriminator c and d
* line 151, 152 modified generators
* line 167, 168 added fake_c and d
* line 286 and 287 added batches C and D
* line 299-302 added imgs_c, imgs_d
* line 310, 311 fake_c and d
* line 315, 316 added reconstr_c and d
* line 324 added img_c and d for g_loss
* line 326-332 added dC and dD loss real and fake
* line 335 modified d_loss
* line 376-384 added fake_c and d, reconstr_c and d
* line 394 added imgs_C, fake_D, reconstr_C, imgs_D, fake_C, reconstr_D to gen_imgs


In [ ]:
!pip install -U tensorflow
!python -m pip show tensorflow

Name: tensorflow
Version: 2.15.0.post1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [ ]:
 ! pip install --upgrade tensorflow

In [ ]:
  !python -m pip show tensorflow

Name: tensorflow
Version: 2.15.0.post1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [ ]:

!apt-get install python-pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python-pip is already the newest version (20.3.4+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
!pip install python-dateutil
!pip install tensorflow-addons

In [ ]:
!pip install -q tensorflow-addons

!pip install --upgrade tensorflow

In [ ]:
      # !pip install -U dateutil

In [ ]:

from __future__ import print_function, division

import datetime
import os

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam as legacy_optimizers
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.losses import Huber
from tensorflow.keras.layers import LayerNormalization
# from dateutil import Dateutils


import matplotlib.pyplot as plt
import numpy as np
import keras.backend as K
from keras import Input, Model
from keras.layers import Conv1D, UpSampling1D, LeakyReLU, Dropout, Lambda, Embedding, Bidirectional, LSTM, Dense, Flatten, Layer, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import legacy as legacy_optimizers
from keras_contrib.layers import InstanceNormalization
#from keras_self_attention import ScaledDotProductAttention
#from keras_self_attention.backend import regularizers
from sklearn.preprocessing import scale

import tensorflow as tf
from tensorflow.python.keras import Sequential
# from tensorflow.python.keras.layers import Normalization
from tensorflow.keras.layers import Normalization

#from Utils.DataUtils import DataUtils





import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
import datetime
import os
!pip install tensorflow-addons
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam as legacy_optimizers
from tensorflow.keras.layers import LayerNormalization
# from tensorflow.keras.losses import huber_loss
from tensorflow.keras.layers import LayerNormalization
# from dateutils import Dateutils

# class TransformerBlock(Layer):
#     def _init_(self, embed_dim, num_heads, rate=0.1):
#         super(TransformerBlock, self)._init_()
#         self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         # self.ffn = Sequential(
#         #     [Dense(ff_dim, activation="relu"), Dense(embed_dim), ]
#         # )
#         #print("apply layered normalization")
#         self.layernorm1 = LayerNormalization(epsilon=1e-4)
#         # self.layernorm2 = LayerNormalization(epsilon=1e-6)
#         #print("apply dropout")
#         self.dropout1 = Dropout(rate)
#         # self.dropout2 = Dropout(rate)

#     def call(self, inputs):
#         #print("In call function of transformer")
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output)
#         out1 = self.layernorm1(inputs * attn_output)
#         # ffn_output = self.ffn(out1)
#         # ffn_output = self.dropout2(ffn_output)
#         return out1
#     def get_config(self):
#         """
#             For rebuilding models on load time.
#         """

#         #config = {
#         #    'output_dim': self.output_dim,
#         #    'units': self.units,
#         #    'return_probabilities': self.return_probabilities
#         #}
#         base_config = super(TransformerBlock, self).get_config()
#         return dict(list(base_config.items()))

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # self.ffn = Sequential(
        #     [Dense(ff_dim, activation="relu"), Dense(embed_dim), ]
        # )
        #print("apply layered normalization")
        self.layernorm1 = LayerNormalization(epsilon=1e-4)
        # self.layernorm2 = LayerNormalization(epsilon=1e-6)
        #print("apply dropout")
        self.dropout1 = Dropout(rate)
        # self.dropout2 = Dropout(rate)

    def call(self, inputs):
        #print("In call function of transformer")
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs * attn_output)
        # ffn_output = self.ffn(out1)
        # ffn_output = self.dropout2(ffn_output)
        return out1

    def get_config(self):
        """
            For rebuilding models on load time.
        """

        #config = {
        #    'output_dim': self.output_dim,
        #    'units': self.units,
        #    'return_probabilities': self.return_probabilities
        #}
        base_config = super(TransformerBlock, self).get_config()
        return dict(list(base_config.items()))



def batch_creation(x_train, y_train, batch_size, batch_idx):
    # Implement your batch creation logic here based on x_train and y_train
    # Return two batches for source and target domains

    batchA = x_train[batch_idx*batch_size:batch_idx*batch_size+batch_size]
    batchB = y_train[batch_idx*batch_size:batch_idx*batch_size+batch_size]
    batchC = y_train[batch_idx*batch_size:batch_idx*batch_size+batch_size]
    batchD = y_train[batch_idx*batch_size:batch_idx*batch_size+batch_size]
    # return batchA, batchB, batchC, batchD
    return batchA, batchB, batchC, batchD


class CycleGAN:
    def __init__(self, row, col):
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                # Restrict TensorFlow to only use the fourth GPU
                tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
        # Input shape
        self.dataUtils = DataUtils()
        self.img_rows = row
        self.img_cols = col
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols)
        self.img_shape_b = (self.img_rows,1)
        self.img_shape_c = (self.img_rows,1)
        self.img_shape_d = (self.img_rows,1)

        # Configure data loader
        self.dataset_name = 'ECG2FECG'

        print("1")
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 4)
        self.disc_patch = (self.img_rows, 1)

        # Number of filters in the first layer of G and D
        self.gf = 6
        self.df = 12

        # Loss weights
        self.lambda_cycle = 4.0  # Cycle-consistency loss
        self.lambda_id = 0.01 * self.lambda_cycle  # Identity loss

        # optimizer = Adam()
        optimizer = tf.keras.optimizers.legacy.Adam()

        # Build and compile the discriminators
        self.d_A = self.build_discriminator(self.img_shape)
        self.d_B = self.build_discriminator(self.img_shape_b)
        self.d_C = self.build_discriminator(self.img_shape_c)
        self.d_D = self.build_discriminator(self.img_shape_d)

        # Build the generators
        self.g_AB = self.build_generator(self.img_shape)
        self.g_BA = self.build_generator(self.img_shape_b)

        # Compile discriminators
        self.d_A.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_C.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_D.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape_b)
        img_D = Input(shape=self.img_shape)
        img_C = Input(shape=self.img_shape_b)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        fake_D = self.g_AB(img_B)
        fake_C = self.g_BA(img_A)

        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        reconstr_C = self.g_BA(fake_D)
        reconstr_D = self.g_AB(fake_C)
        #print("1")
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)
        img_D_id = self.g_BA(img_D)
        img_C_id = self.g_AB(img_C)

        #print("2")
        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False
        #print("3")

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)
        # valid_C = self.d_C.train_on_batch(fake_C, valid)
        # valid_D = self.d_C.train_on_batch(fake_d, valid)
        valid_C = self.d_C(fake_C)
        valid_D = self.d_C(fake_D)
        print("4")

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B, img_C, img_D],
                              outputs=[valid_A, valid_B, valid_C, valid_D,
                                       fake_B, fake_A, fake_D, fake_C,
                                       reconstr_A, reconstr_B, reconstr_C, reconstr_D])
        print("model created")
        # self.combined.compile(loss=['huber_loss', 'huber_loss',
        #                             'huber_loss', 'huber_loss', 'huber_loss',
        #                             'huber_loss'],
        #                       loss_weights=[1, 1,
        #                                     self.lambda_cycle, self.lambda_cycle,
        #                                     self.lambda_id, self.lambda_id],
        #                       optimizer=optimizer)

        self.combined.compile(loss=[Huber(), Huber(),
                            Huber(), Huber(), Huber(),
                            Huber()],
                      loss_weights=[1, 1,
                                    self.lambda_cycle, self.lambda_cycle,
                                    self.lambda_id, self.lambda_id],
                      optimizer=optimizer)

        print("Model compiled")


    # Define custom loss
    def custom_loss(self):

        # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
        def loss(y_true, y_pred):
            return K.mean(y_true * K.log(y_true / y_pred + K.epsilon()))

        # Return a function
        return loss

    def build_generator(self,img_shape):
        """U-Net Generator"""
        print("In build generator")
        def conv1DWithSINE(layer_input, filters, f_size=30):
            """Layers used during downsampling"""
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='sigmoid')(layer_input)
            d = InstanceNormalization()(d)
            return d

        def multiply(x):
            mask,image  = x
            return image* K.clip(mask,0.8,1)

        input = Input(shape=img_shape)
        print("Input shape:: ", input.shape)
        value = conv1DWithSINE(input, input.shape[2], f_size=30)
        print(" shape after conv1d :: ", value.shape)

        print("apply attention.")
        att = TransformerBlock(embed_dim=input.shape[1], num_heads=2)(value)
        print("shape after attention:: ",att.shape )
        att = Normalization(axis=1)(att)
        print("shape after normalizing attention:: ",att.shape )

        remainedInput = Lambda(multiply)([att, value])
        print("shape after apply lambda:: ",remainedInput.shape )

        output_img = conv1DWithSINE(remainedInput, 13, f_size=13)
        output_img = conv1DWithSINE(output_img, 13, f_size=13)
        output_img = conv1DWithSINE(output_img, 13, f_size=13)
        output_img = conv1DWithSINE(output_img, 1, f_size=1)

        return Model(input, output_img)

    def build_discriminator(self,img_shape):
        print("In building discriminator")
        def d_layer(layer_input, filters, f_size=13, normalization=True):
            """Discriminator layer"""
            d = Conv1D(filters, kernel_size=f_size, padding='same',activation='sigmoid')(layer_input)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=img_shape)
        print("input shape:: ",img.shape)
        d1 = d_layer(img, self.df)
        print("input shape after first conv1d:: ",d1.shape)
        d2 = d_layer(d1, 7)
        print("input shape after second conv1d:: ",d2.shape)
        d3 = d_layer(d2, 5)
        print("input shape after third conv1d:: ",d3.shape)
        validity = d_layer(d3, 3)
        # validity = d_layer(d3, img.shape[1])
        print("final shape after 4th layer:: ",validity.shape)


        return Model(img, validity)

    def train(self, x_train, y_train, epochs, batch_size=10, sample_interval=20):
        print("Num Samples", x_train.shape[0])
        print("In train:: ")
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_idx in range(x_train.shape[0] // batch_size):
                batchA, batchB, batchC, batchD = batch_creation(x_train, y_train, batch_size, batch_idx)
                batchA = np.asarray(batchA)
                batchB = np.asarray(batchB)
                batchC = np.asarray(batchC)
                batchD = np.asarray(batchD)

                imgs_A = batchA
                imgs_B = batchB
                imgs_C = batchC
                imgs_D = batchD
                imgs_B = np.reshape(imgs_B, (-1, x_train.shape[1], x_train.shape[2]))
                imgs_A = np.reshape(imgs_A, (-1, x_train.shape[1], x_train.shape[2]))
                imgs_D = np.reshape(imgs_D, (-1, x_train.shape[1], x_train.shape[2]))
                imgs_C = np.reshape(imgs_C, (-1, x_train.shape[1], x_train.shape[2]))
                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to the other domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                fake_D = self.g_AB.predict(imgs_C)
                fake_C = self.g_BA.predict(imgs_D)

                # Translate images back to original domain
                reconstr_A = self.g_BA.predict(fake_B)
                reconstr_B = self.g_AB.predict(fake_A)
                reconstr_C = self.g_BA.predict(fake_D)
                reconstr_D = self.g_AB.predict(fake_C)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                dC_loss_real = self.d_C.train_on_batch(imgs_C, valid)
                dC_loss_fake = self.d_C.train_on_batch(fake_C, fake)
                dC_loss = 0.5 * np.add(dC_loss_real, dC_loss_fake)

                dD_loss_real = self.d_D.train_on_batch(imgs_D, valid)
                dD_loss_fake = self.d_D.train_on_batch(fake_D, fake)
                dD_loss = 0.5 * np.add(dD_loss_real, dD_loss_fake)

                # Total discriminator loss
                # d_loss = 0.25 * np.add(dA_loss, dB_loss, dC_loss, dD_loss)
                # Total discriminator loss
                d_loss = 0.25 * (dA_loss + dB_loss + dC_loss + dD_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators

                # g_loss = self.combined.train_on_batch([imgs_A, imgs_B, imgs_C, imgs_D],
                #                                       [valid, valid, valid, valid,
                #                                        imgs_B, imgs_A, imgs_D, imgs_C,
                #                                        reconstr_A, reconstr_B,
                #                                        reconstr_C, reconstr_D])
                # g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                #                                       [valid, valid,
                #                                        imgs_B, imgs_A,
                #                                        reconstr_A, reconstr_B])
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B, imgs_C, imgs_D],
                                      [valid, valid, valid, valid,
                                       imgs_B, imgs_A, imgs_D, imgs_C,
                                       reconstr_A, reconstr_B,
                                       reconstr_C, reconstr_D])


                # # Translate images to the other domain
                # fake_B = self.g_AB.predict(imgs_A)
                # fake_A = self.g_BA.predict(imgs_B)
                # # Translate images back to original domain
                # reconstr_A = self.g_BA.predict(fake_B)
                # reconstr_B = self.g_AB.predict(fake_A)
                # # Train the discriminators (original images = real / translated = Fake)
                # dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                # dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                # dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                # dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                # dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                # dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # # Total disciminator loss
                # d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # # ------------------
                # #  Train Generators
                # # ------------------


                # # Train the generators
                # g_loss = self.combined.train_on_batch([imgs_A, imgs_B, imgs_C, imgs_D],
                #                                       [valid, valid,
                #                                        imgs_B, imgs_A,
                #                                        reconstr_A, reconstr_B,
                #                                        valid_C, valid_D])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                      % (epoch, epochs,
                         batch_idx, 1,
                         d_loss[0], 100 * d_loss[1],
                         g_loss[0],
                         np.mean(g_loss[1:3]),
                         np.mean(g_loss[3:5]),
                         np.mean(g_loss[5:6]),
                         elapsed_time))

                # If at save interval => save generated image samples
                #print("shape of image a",imgs_A.shape)
                #print("shape of image b",imgs_B.shape)
                if batch_idx % sample_interval == 0:
                    self.sample_images(epoch, batch_idx, imgs_A, imgs_B, imgs_C, imgs_D)
                    self.g_AB.save("ECG2FECG.h5", overwrite=True)
                    self.g_BA.save("FECG2ECG.h5", overwrite=True)

    def sample_images(self, epoch, batch_idx, imgs_A, imgs_B, imgs_C, imgs_D):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        # Demo (for GIF)
        # imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        # imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        fake_D = self.g_AB.predict(imgs_B)
        fake_C = self.g_BA.predict(imgs_A)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)
        reconstr_C = self.g_BA.predict(fake_D)
        reconstr_D = self.g_AB.predict(fake_C)


        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B, imgs_C, fake_D, reconstr_C, imgs_D, fake_C, reconstr_D])
        #print("shape of gen",gen_imgs.shape)
        # Rescale images 0 - 1
        # gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0

        # gen_imgs[1] = gen_imgs[0]-gen_imgs[1]
        try:
            for i in range(r):
                for j in range(c):
                    for bias in range(1):
                        #gen_imgs[cnt][:, bias] = scale(self.dataUtils.butter_bandpass_filter(gen_imgs[cnt][:, bias], 10, 50, 200, axis=0), axis=0)
                        #gen_imgs[cnt][:, bias] = scale(gen_imgs[cnt][:, bias], axis=0)
                        if np.max(gen_imgs[cnt][:, bias]) != 0:
                            gen_imgs[cnt][:, bias] = gen_imgs[cnt][:, bias] / np.max(gen_imgs[cnt][:, bias])
                        axs[i, j].plot(gen_imgs[cnt][:, bias] + bias)
                    axs[i, j].set_title(titles[j])
                    cnt += 1
            # fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_idx))
            plt.close()
        except:
            pass



In [ ]:
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers import TimeDistributed
#from keras.layers.recurrent import Recurrent
from tensorflow.keras.layers import InputSpec




class AttentionDecoder():

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space
        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units,),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units,),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units,),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim,),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def _time_distributed_dense(self,x, w, b=None, dropout=None,
                                input_dim=None, output_dim=None,
                                timesteps=None, training=None):
        """Apply `y . w + b` for every temporal slice y of x.
        # Arguments
            x: input tensor.
            w: weight matrix.
            b: optional bias vector.
            dropout: wether to apply dropout (same dropout mask
                for every temporal slice of the input).
            input_dim: integer; optional dimensionality of the input.
            output_dim: integer; optional dimensionality of the output.
            timesteps: integer; optional number of timesteps.
            training: training phase tensor or boolean.
        # Returns
            Output tensor.
        """
        if not input_dim:
            input_dim = K.shape(x)[2]
        if not timesteps:
            timesteps = K.shape(x)[1]
        if not output_dim:
            output_dim = K.shape(w)[1]

        if dropout is not None and 0. < dropout < 1.:
            # apply the same dropout pattern at every timestep
            ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
            dropout_matrix = K.dropout(ones, dropout)
            expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
            x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)

        # collapse time dimension and batch dimension together
        x = K.reshape(x, (-1, input_dim))
        x = K.dot(x, w)
        if b is not None:
            x = K.bias_add(x, b)
        # reshape to 3D tensor
        if K.backend() == 'tensorflow':
            x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
            x.set_shape([None, None, output_dim])
        else:
            x = K.reshape(x, (-1, timesteps, output_dim))
        return x

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = self._time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x, )

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1 - zt) * stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
!pip install utils

In [ ]:
# import numpy as np


# # See https://github.com/keras-rl/keras-rl/blob/master/rl/memory.py
# def zeros_like(a, dtype='float32'):
#     """Return an array of zeros with same shape as given array.

#     Args:
#         a (array_like, iterable): An object with shape attribute or an iterable.

#     Returns:
#         (array_like, list): Array of zeros with the same shape as a.
#     """
#     if hasattr(a, 'shape'):
#         if hasattr(a, 'dtype'):
#             dtype = a.dtype
#         return np.zeros(a.shape, dtype=dtype)
#     if hasattr(a, '__iter__'):
#         return [zeros_like(b, dtype=dtype) for b in a]
#     return 0.



# def check_shape(a, b):
#     """Check if the shapes of given values match.

#     Args:
#         a (array_like, tuple): An object with shape attribute or a tuple representing shape.
#         b (array_like, tuple): An object with shape attribute or a tuple representing shape.

#     Raises:
#         Exception: When shapes don't match.
#     """
#     if hasattr(a, 'shape'):
#         a = a.shape
#     if hasattr(b, 'shape'):
#         b = b.shape
#     assert a == b, f"Shapes {a} and {b} don't match"



# def unique(a):
#     res = {id(v): v for v in a}
#     return list(res.values())


In [ ]:

from unittest import TestCase
import numpy as np

# from utils.TrainUtils import TrainUtil
# from utils.TrainUtils import TrainUtils
# from deeplearning.CycleGAN import CycleGAN


class TestCycleGAN(TestCase):

    def __init__(self, methodName: str = ...) -> None:
        super().__init__(methodName)
        self.trainUtils = TrainUtils()

    def test_trainSignal(self,path="/content/drive/MyDrive/drive-download-20230428T081947Z-001 (1)/b010.edf"):
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2,path="/content/drive/MyDrive/drive-download-20230428T081947Z-001 (1)/")
        print("ecgWindows:: ",np.array(ecgWindows).shape)
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        X_train = np.reshape(X_train, [-1, X_train.shape[1], X_train.shape[2]])
        # X_test = np.reshape(X_test, [-1, X_test.shape[1], X_test.shape[2], 1])
        Y_train = np.reshape(Y_train, [-1, Y_train.shape[1], Y_train.shape[2]])

        print("Shape of x train and y train:: ",X_train.shape, Y_train.shape)
        print("Shape of x test and y test:: ",X_test.shape, y_test.shape)
        # y_test = np.reshape(Y_test, [-1, Y_test.shape[1], Y_test.shape[2], 1])
        cycleGAN = CycleGAN(X_train.shape[1], X_train.shape[2])
        print("model instantiated")
        cycleGAN.train(x_train=X_train, y_train=Y_train, epochs=1)
        return cycleGAN, X_test, y_test
    def divide_test_train(self,path="/content/drive/MyDrive/drive-download-20230428T081947Z-001 (1)/b010.edf"):
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2,path="/content/drive/MyDrive/drive-download-20230428T081947Z-001 (1)")
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        return X_train, X_test, Y_train, y_test
#model = TestCycleGAN("test_trainSignal")

In [ ]:
model = TestCycleGAN("test_trainSignal")

In [ ]:
cycleGan, X_test, y_test = model.test_trainSignal()

Reading file::  /content/drive/MyDrive/drive-download-20230428T081947Z-001 (1)/b010.edf
different columns::  ['I', 'II', 'RESP', 'SCG']
total number of samples [1505000 1505000 1505000 1505000]
before downsampling 1505000
after downsampling 301000
(1, 301000)
ECG all merged shape::  (1, 301000)
SCG shape after windowing::  (300, 1000, 1)
ECG shape after windowing::  (300, 1000, 1)
ecgWindows::  (300, 1000, 1)
Splitting into train and test:: 
Shape of x train and y train::  (225, 1000, 1) (225, 1000, 1)
Shape of x test and y test::  (75, 1000, 1) (75, 1000, 1)
1 Physical GPUs, 1 Logical GPUs
1
In building discriminator
input shape::  (None, 1000, 1)
input shape after first conv1d::  (None, 1000, 12)
input shape after second conv1d::  (None, 1000, 7)
input shape after third conv1d::  (None, 1000, 5)
final shape after 4th layer::  (None, 1000, 3)
In building discriminator
input shape::  (None, 1000, 1)
input shape after first conv1d::  (None, 1000, 12)
input shape after second conv1d::  (

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


4
model created
Model compiled
model instantiated
Num Samples 225
In train:: 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 627ms/step


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 75ms/step


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[Epoch 0/1] [Batch 0/1] [D loss: 1.485881, acc:  31%] [G loss: 7.431767, adv: 0.734807, recon: 0.738422, id: 0.660117] time: 0:00:35.335130 
1/1 [==============================] - 0s 40ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  `tf.saved_model.SaveOptions` object that specifies SavedModel


1/1 [==============================] - 0s 41ms/step
[Epoch 0/1] [Batch 1/1] [D loss: 1.475348, acc:  35%] [G loss: 7.295848, adv: 0.733102, recon: 0.722163, id: 0.606764] time: 0:00:37.307636 
1/1 [==============================] - 0s 40ms/step
[Epoch 0/1] [Batch 2/1] [D loss: 1.462747, acc:  44%] [G loss: 7.133142, adv: 0.749021, recon: 0.697957, id: 0.584619] time: 0:00:38.421275 
1/1 [==============================] - 0s 42ms/step
[Epoch 0/1] [Batch 3/1] [D loss: 1.448382, acc:  41%] [G loss: 7.064125, adv: 0.725170, recon: 0.695209, id: 0.563141] time: 0:00:39.546734 
1/1 [==============================] - 0s 41ms/step
[Epoch 0/1] [Batch 4/1] [D loss: 1.442062, acc:  39%] [G loss: 7.062830, adv: 0.711589, recon: 0.698444, id: 0.555235] time: 0:00:40.646249 
1/1 [==============================] - 0s 43ms/step
[Epoch 0/1] [Batch 5/1] [D loss: 1.436016, acc:  37%] [G loss: 7.095674, adv: 0.718971, recon: 0.700845, id: 0.540625] time: 0:00:41.770045 
1/1 [==============================

1/1 [==============================] - 0s 57ms/step
[Epoch 0/1] [Batch 21/1] [D loss: 1.232829, acc:  37%] [G loss: 5.752021, adv: 0.625921, recon: 0.556291, id: 0.571511] time: 0:01:03.867649 


In [ ]:
#model = TestcycleGAN("test_trainSignal")

In [ ]:
#cycleGan, X_test, y_test = model.test_trainSignal()

In [ ]:
x_g_AB =cycleGan.g_AB.predict(X_test)
#x_g_BA = cycleGan.g_BA.predict(X_test)

In [ ]:
# import pandas as pd
# import numpy as np

# a = np.array(y_test.flatten())
# b = x_g_AB.flatten()
# print(len(a), len(b))
# df1 = pd.DataFrame({ "ECG Original" : a})
# df2 = pd.DataFrame({ "ECG Predicted" : b})
# df1.to_csv("/content/drive/MyDrive/ECG_data/original_ECG_1.csv", index=False)
# df2.to_csv("/content/drive/MyDrive/ECG_data/generated_ECG_1.csv", index=False)




import pandas as pd
import numpy as np

# a = np.array(original_scg)
a = np.array(y_test.flatten())
b = x_g_AB.flatten()
print(len(a), len(b))
df1 = pd.DataFrame({ "ECG Original" : a})
df2 = pd.DataFrame({ "ECG Predicted" : b})
df1.to_csv("original_ECG_10.csv", index=False)
#df2.to_csv("/content/drive/MyDrive/ECG_data/generated_ECG_10.csv", index=False)


In [ ]:
!pip install heartpy

In [ ]:
import heartpy as hp
import matplotlib.pyplot as plt

sample_rate = 250

In [ ]:
data = hp.get_data('/content/original_ECG_10.csv')

plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()

In [ ]:
#run analysis
wd, m = hp.process(data[1:], sample_rate)

#visualise in plot of custom size
plt.figure(figsize=(12,4))
hp.plotter(wd, m)
#print(len(wd['hr']))
#print(wd.keys())
#print(wd['RR_list'])
#heart rates
hrs = 60000/wd['RR_list']
print("heart rates: ",hrs)
#display computed measures
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [ ]:

df2 = pd.DataFrame({ "Heart Rate" : np.array(hrs)})
df2.to_csv("hr_original_10.csv", index=False)


In [ ]:
data = hp.get_data('original_ECG_15.csv')

plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()

In [ ]:
#run analysis
wd, m = hp.process(data[1:], sample_rate)

#visualise in plot of custom size
plt.figure(figsize=(12,4))
hp.plotter(wd, m)
#print(len(wd['hr']))
#print(wd.keys())
#print(wd['RR_list'])
#heart rates
hrs = 60000/wd['RR_list']
print("heart rates: ",hrs)
#display computed measures
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [ ]:

df2 = pd.DataFrame({ "Heart Rate" : np.array(hrs)})
df2.to_csv("/content/drive/MyDrive/ECG_data/hr_original_12.csv", index=False)


In [ ]:
data = hp.get_data('/content/drive/MyDrive/ECG_data/hr_original_10.csv')

In [ ]:
for i in range(len(data)):
  if data[i]>500:
    data[i]=data[i]/10

In [ ]:
import pandas as pd
import numpy as np
df2 = pd.DataFrame({ "Heart Rate" : np.array(data)})
df2.to_csv("/content/drive/MyDrive/ECG_data/hr_original_10.csv", index=False)


In [ ]:
plt.plot(x_g_AB[1])
plt.plot(y_test[1])

In [ ]:
#X_test_in = []
#x_g_AB_in = []
#inverse differencing
#for i in range(len(X_test)):
#  X_test_in.append((np.concatenate(([X_test[i][0]], X_test[i])).cumsum())[:x_g_AB.shape[1]])
#  x_g_AB_in.append((np.concatenate(([x_g_AB[i][0]], x_g_AB[i])).cumsum())[:x_g_AB.shape[1]])

In [ ]:
len(x_g_AB_in)

In [ ]:
len(x_g_AB_in[0])

In [ ]:
# x_g_AB = np.array(x_g_AB_in).reshape(x_g_AB.shape[0],x_g_AB.shape[1],1)
# X_test = np.array(X_test_in).reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
def smooth(x,window_len=7,window='hanning'):
    """smooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if window_len<3:
        return x

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


In [ ]:
x_g_AB_reshaped = x_g_AB.reshape(x_g_AB.shape[0],x_g_AB.shape[1])

In [ ]:
x_g_AB_reshaped.shape

In [ ]:
x_g_AB_smoothed = []
for i in range(len(x_g_AB_reshaped)):
  smoothed = smooth(x_g_AB_reshaped[i])
  x_g_AB_smoothed.append(smoothed[:len(x_g_AB_reshaped[i])])


In [ ]:
#          Testing functions
#################################################################################
def rmse(targets, predictions):
    return np.sqrt(np.mean((targets-predictions)**2))


def prd(targets, predictions):
    s1 = np.sum((targets-predictions)**2)
    s2 = np.sum(targets**2)
    return np.sqrt(s1 / s2 * 100)


def mmd(targets, predictions):
    mmd_stat = MMDStatistic(400, 400)
    sample_target = torch.from_numpy(targets.numpy().reshape((400,1)))
    sample_pred = torch.from_numpy(predictions.numpy().reshape((400,1)))

    stat = mmd_stat(sample_target, sample_pred, [1.])
    return(stat.item())


In [ ]:
x_g_AB_smoothed = np.array(x_g_AB_smoothed)
x_g_AB_smoothed = x_g_AB_smoothed.reshape(x_g_AB_smoothed.shape[0],x_g_AB_smoothed.shape[1],1)

In [ ]:
y_test.shape

In [ ]:
x_g_AB_smoothed.shape

In [ ]:
#for i in range(len(x_g_AB)):
rmse_arr = rmse(y_test,x_g_AB)
print(rmse_arr)
rmse_smoothed = rmse(y_test,x_g_AB_smoothed)
print("smoothed",rmse_smoothed)

In [ ]:
rmse_all =[]
rmse_all_smoothened =[]
for i in range(len(x_g_AB)):
  rmse_all.append(rmse(y_test[i],x_g_AB[i]))
  rmse_all_smoothened.append(rmse(y_test[i],x_g_AB_smoothed[i]))

rmse_all_mean = np.array(rmse_all).mean()
print(rmse_all_mean)
rmse_all_smoothed_mean = np.array(rmse_all_smoothened).mean()
print("smoothed",rmse_all_smoothed_mean)

In [ ]:

prd_arr = prd(y_test,x_g_AB)
print(prd_arr)
prd_arr_smoothed = prd(y_test,x_g_AB_smoothed)
print("smoothed",prd_arr_smoothed)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae_arr = mean_absolute_error(y_test.flatten(),x_g_AB.flatten())
print(mae_arr)
mae_arr_smoothed = mean_absolute_error(y_test.flatten(),x_g_AB_smoothed.flatten())
print("smoothed",mae_arr_smoothed)

In [ ]:
def detect_peaks(ecg_signal, threshold=0.3, qrs_filter=None):
    '''
    Peak detection algorithm using cross corrrelation and threshold
    '''
    if qrs_filter is None:
        # create default qrs filter, which is just a part of the sine function
        t = np.linspace(1.5 * np.pi, 3.5 * np.pi, 15)
        qrs_filter = np.sin(t)

    # normalize data
    ecg_signal = (ecg_signal - ecg_signal.mean()) / ecg_signal.std()

    # calculate cross correlation
    similarity = np.correlate(ecg_signal, qrs_filter, mode="same")
    similarity = similarity / np.max(similarity)

    # return peaks (values in ms) using threshold
    return ecg_signal[similarity > threshold].index, similarity

In [ ]:
def group_peaks(p, threshold=5):
    '''
    The peak detection algorithm finds multiple peaks for each QRS complex.
    Here we group collections of peaks that are very near (within threshold) and we take the median index
    '''
    # initialize output
    output = np.empty(0)

    # label groups of sample that belong to the same peak
    peak_groups, num_groups = label(np.diff(p) < threshold)

    # iterate through groups and take the mean as peak index
    for i in np.unique(peak_groups)[1:]:
        peak_group = p[np.where(peak_groups == i)]
        output = np.append(output, np.median(peak_group))
    return output

In [ ]:
import pandas as pd
df = pd.DataFrame(x_g_AB.flatten(), columns = ['ECG'])
df2 = pd.DataFrame(X_test.flatten(), columns = ['SCG'])
df3 = pd.DataFrame(y_test.flatten(), columns = ['ECG_orig'])
#df['column_name']=pd.Series(x_g_AB.flatten())

In [ ]:
n_samples = f.getNSamples()[0]

# Extract the signal data and flatten it
signal_data = f.readSignal(0).flatten()

# Close the file
f.close()

In [ ]:
!pip install wfdb

In [ ]:
import wfdb
from wfdb import processing

In [ ]:
# to be used for wsdb to edf conversion and reading data using wsdb
import wfdb
from google.colab import files
fileName_str
record = wfdb.rdrecord('b001', pn_dir='cebsdb')

In [ ]:
# Get the number of channels and samples
n_channels = f.signals_in_file
n_samples = f.getNSamples()[0]

# Extract the signal data and flatten it
signal_data = f.readSignal(0).flatten()

# Close the file
f.close()

In [ ]:
record.p_signal[0]

In [ ]:
qrs = codeFn(ecg_signal, fs)

In [ ]:

# Demo 19 - Use the GQRS detection algorithm and correct the peaks

def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    print(hrs)
    N = sig.shape[0]

    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()

    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x',
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()
    return hrs



In [ ]:
t = np.arange(len(ecg_signal)) / fs
plt.plot(t, ecg_signal)
plt.plot(qrs/fs, ecg_signal[qrs], 'ro')
plt.xlabel('Time (s)')
plt.ylabel('ECG Amplitude')
plt.show()

In [ ]:
# Load the WFDB record and the physical samples
#record = wfdb.rdrecord('sample-data/100', sampfrom=0, sampto=10000, channels=[0])

# Use the GQRS algorithm to detect QRS locations in the first channel
qrs_inds = processing.qrs.gqrs_detect(sig=record.p_signal[:,0], fs=record.fs)

# Plot results
hrs = peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
         title="GQRS peak detection on record b001")

# Correct the peaks shifting them to local maxima
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.peaks.correct_peaks(record.p_signal[:,0],
                                                     peak_inds=qrs_inds,
                                                     search_radius=search_radius,
                                                     smooth_window_size=150)

# Display results
print('Corrected GQRS detected peak indices:', sorted(corrected_peak_inds))
hrs = peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/100")


In [ ]:
# Load the WFDB record and the physical samples
#record = wfdb.rdrecord('sample-data/100', sampfrom=0, sampto=10000, channels=[0])

# Use the GQRS algorithm to detect QRS locations in the first channel
qrs_inds = processing.qrs.gqrs_detect(sig=record.p_signal[:,3], fs=record.fs)

# Plot results
hrs_scg = peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
         title="GQRS peak detection on record b001")

# Correct the peaks shifting them to local maxima
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.peaks.correct_peaks(record.p_signal[:,3],
                                                     peak_inds=qrs_inds,
                                                     search_radius=search_radius,
                                                     smooth_window_size=150)

# Display results
print('Corrected GQRS detected peak indices:', sorted(corrected_peak_inds))
hrs_scg = peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/100")


In [ ]:
len(hrs)
hrs = hrs[~np.isnan(hrs)]
hrs=hrs[hrs<160]
#hrs=hrs[hrs>100]
print(len(hrs))

In [ ]:
len(hrs_scg)

hrs_scg = hrs_scg[~np.isnan(hrs_scg)]
hrs_scg=hrs_scg[hrs_scg<160]
#hrs_scg=hrs_scg[hrs_scg>100]
print(len(hrs_scg))

In [ ]:
# plotting the data
plt.scatter(hrs[:len(hrs_scg)], hrs_scg)

# This will fit the best line into the graph
plt.plot(np.unique(hrs), np.poly1d(np.polyfit(hrs[:len(hrs_scg)], hrs_scg, 1))
         (np.unique(hrs)), color='red')
plt.ylabel("SCG heart rate")
plt.xlabel("ECG heart rate")

In [ ]:
!pip install pyhrv

In [ ]:
# detect peaks
print(len(df.ECG))
print(len(df2.SCG))
print(len(df3.ECG_orig))
peaks_ecg_pred, similarity = detect_peaks(df.ECG, threshold=0.3)
peaks_scg, similarity = detect_peaks(df2.SCG, threshold=0.3)
peaks_ecg, similarity = detect_peaks(df3.ECG_orig, threshold=0.3)

print(len(peaks_ecg_pred))
print(len(peaks_scg))
print(len(peaks_ecg))
# group peaks so we get a single peak per beat (hopefully)

from scipy.ndimage import label
grouped_peaks_ecg_pred = group_peaks(peaks_ecg_pred)
grouped_peaks_scg = group_peaks(peaks_scg)
grouped_peaks_ecg = group_peaks(peaks_ecg)

print(len(grouped_peaks_ecg_pred))
print(len(grouped_peaks_scg))
print(len(grouped_peaks_ecg))
# RR-intervals are the differences between successive peaks
rr_ecg_pred = np.diff(grouped_peaks_ecg_pred)
rr_scg = np.diff(grouped_peaks_scg)
rr_ecg = np.diff(grouped_peaks_ecg)

print(len(rr_ecg_pred))
print(len(rr_scg))
print(len(rr_ecg))
# plot RR-intervals
#plt.figure(figsize=(20, 7))
#plt.title("RR-intervals")
#plt.xlabel("Time (ms)")
#plt.ylabel("RR-interval (ms)")

#plt.plot(np.cumsum(rr), rr, label="RR-interval", color="#A651D8")
#plt.show()

In [ ]:

from matplotlib.patches import Ellipse
import seaborn as sns

In [ ]:
def plot_poincare(rr):
    rr_n = rr[:-1]
    rr_n1 = rr[1:]

    sd1 = np.sqrt(0.5) * np.std(rr_n1 - rr_n)
    sd2 = np.sqrt(0.5) * np.std(rr_n1 + rr_n)

    m = np.mean(rr)
    min_rr = np.min(rr)
    max_rr = np.max(rr)

    plt.figure(figsize=(10, 10))
    plt.title("Poincare plot")

    sns.scatterplot(x=rr_n, y=rr_n1, color="#51A6D8")

    plt.xlabel(r'$RR_n (ms)$')
    plt.ylabel(r'$RR_{n+1} (ms)$')

    e1 = Ellipse((m, m), 2*sd1, 2*sd2, angle=-45, linewidth=1.2, fill=False, color="k")
    plt.gca().add_patch(e1)

    plt.arrow(m, m, (max_rr-min_rr)*0.4, (max_rr-min_rr)*0.4, color="k", linewidth=0.8, head_width=5, head_length=5)
    plt.arrow(m, m, (min_rr-max_rr)*0.4, (max_rr-min_rr)*0.4, color="k", linewidth=0.8, head_width=5, head_length=5)

    plt.arrow(m, m, sd2 * np.sqrt(0.5), sd2 * np.sqrt(0.5), color="green", linewidth=5)
    plt.arrow(m, m, -sd1 * np.sqrt(0.5), sd1 * np.sqrt(0.5), color="red", linewidth=5)

    plt.text(max_rr, max_rr, "SD2", fontsize=20, color="green")
    plt.text(m-(max_rr-min_rr)*0.4-20, max_rr, "SD1", fontsize=20, color="red")

    return sd1, sd2

In [ ]:
sd1, sd2 = plot_poincare(rr_ecg_pred)
print("SD1: %.3f ms" % sd1)
print("SD2: %.3f ms" % sd2)

In [ ]:
sd1, sd2 = plot_poincare(rr_scg)
print("SD1: %.3f ms" % sd1)
print("SD2: %.3f ms" % sd2)

In [ ]:
import pyhrv
pyhrv.nonlinear.poincare(rr_ecg_pred,peaks_ecg_pred)

In [ ]:
from scipy.stats import pearsonr
true = y_test.astype('float')
pre =  x_g_AB.astype('float')
corr, pval =pearsonr(true.flatten(),pre.flatten())
print("corre and p-val:: ",corr, pval)


pre_smoothed =  x_g_AB_smoothed.astype('float')
corr_sm, pval_sm =pearsonr(true.flatten(),pre_smoothed.flatten())
print("smoothed corre and p-val:: ",corr_sm, pval_sm)

In [ ]:
accuracy_for_AtoB = 100 - (np.mean(np.abs(x_g_AB-y_test)*100))
print(accuracy_for_AtoB)

accuracy_for_AtoB_sm = 100 - (np.mean(np.abs(x_g_AB_smoothed-y_test)*100))
print("smoothed",accuracy_for_AtoB_sm)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300


In [ ]:
print(len(str(fileNames[0]))-4)

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
plt.plot(y_test.flatten()[:5000],color="blue")
plt.plot(np.array(x_g_AB).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_first_5000.jpg')

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
plt.plot(y_test.flatten()[:5000],color="blue")
# plt.plot(np.array(x_g_AB).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_blue.jpg')

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
# plt.plot(y_test.flatten()[:5000],color="blue")
plt.plot(np.array(x_g_AB).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_red.jpg')

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
plt.plot(y_test.flatten()[:5000],color="blue")
plt.plot(np.array(x_g_AB_smoothed).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_first_5000_smoothed.jpg')

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
plt.plot(y_test.flatten()[:5000],color="blue")
# plt.plot(np.array(x_g_AB_smoothed).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_blue_smoothed.jpg')

In [ ]:
from google.colab import files
fig = plt.figure(figsize=(4,3))
fig.set_size_inches(50,7)
# plt.plot(y_test.flatten()[:5000],color="blue")
plt.plot(np.array(x_g_AB_smoothed).flatten()[:5000], color="red")
plt.show()
fig.savefig('/content/drive/MyDrive/Colab Notebooks/basal/b_to_b_'+str(fileNames[0])[0:len(str(fileNames[0]))-4]+'_red_smoothed.jpg')

In [ ]:
# for i in range(len(y_test)):
#   fig = plt.figure(figsize=(4,3))
#   fig.set_size_inches(50,7)
#   plt.plot(y_test[i],color="blue")
#   plt.plot(x_g_AB[i], color="red")
#   plt.show()
#   plt.savefig('/content/drive/MyDrive/Colab Notebooks/GAN_PAPER_DATA/basal/b_to_b_1_batch_'+str(i+1)+'.jpg')

In [ ]:
# for i in range(len(y_test)):
#   fig = plt.figure(figsize=(4,3))
#   fig.set_size_inches(50,7)
#   plt.plot(y_test[i],color="blue")
#   plt.plot(x_g_AB_smoothed[i], color="red")
#   plt.savefig('/content/drive/MyDrive/Colab Notebooks/GAN_PAPER_DATA/basal/b_to_b_1_smoothed_batch_'+str(i+1)+'.jpg')
#   plt.show()

In [ ]:
# !pip install frechetdist

In [ ]:
# import sys
# print(sys.getrecursionlimit())
# sys.setrecursionlimit(5000)
# print(sys.getrecursionlimit())

In [ ]:
# from frechetdist import frdist
# fd = []
# for i in range(len(y_test)):
#   fd.append(frdist(y_test[i],x_g_AB[i]))
# print("FD: ", np.array(fd).mean())

In [ ]:

#mmd_arr = mmd(y_test,x_g_AB)

In [ ]:
# fd_mean = np.array(fd).mean()

In [ ]:
#To create the blank csv with header.

#from csv import writer

#List=["Subject","Generator param A", "Generator param B","Reconstructor param A","Reconstructor param B","RMSE flattened", "RMSE Smoothed flattened","RMSE All Mean", "RMSE All Smoothed Mean", "prd", "prd smoothed","mae","mae smoothed", "correlation","correlation smoothed","p-val","p-val smoothed","accuracy","accuracy smoothed","frachet dist"]

#with open('/content/drive/MyDrive/Colab Notebooks/GAN_PAPER_DATA/basal/matrices.csv', 'w') as f_object:

#    writer_object = writer(f_object)
#    writer_object.writerow(List)

#    f_object.close()

In [ ]:
# change the parameters here
gen_param_a = 0
gen_param_b = 0
rec_param_a = 0
rec_param_b = 0

In [ ]:
# List
List_vals =[fileNames[0], gen_param_a,gen_param_b,rec_param_a,rec_param_b, rmse_arr, rmse_smoothed, rmse_all_mean, rmse_all_smoothed_mean, prd_arr, prd_arr_smoothed,mae_arr,mae_arr_smoothed, corr,corr_sm,pval,pval_sm,accuracy_for_AtoB,accuracy_for_AtoB_sm]

In [ ]:

from csv import writer

with open('/content/drive/MyDrive/Colab Notebooks/basal/matrices.csv', 'a') as f_object:

    writer_object = writer(f_object)
    writer_object.writerow(List_vals)

    f_object.close()

In [ ]:
# !pip install torch

In [ ]:
# !pip install -c dloewenstein torch-two-sample

In [ ]:
#!pip install mmd

In [ ]:
# !pip install torch torchvision



In [ ]:
# from torch_two_sample.statistics_diff import MMDStatistic
# print(mmd(y_test,x_g_AB))

In [ ]:
#!pip install ecg_plot

In [ ]:
#import ecg_plot

In [ ]:
#for i in range(10):
#  ecg_plot.plot_1(y_test_scaled[i], sample_rate=500, title = 'Original ECG')
 # ecg_plot.plot_1(x_g_AB_scaled[i], sample_rate=500, title = 'Predicted ECG')
  #plt.tight_layout()
 # ecg_plot.show()

In [ ]:

#ecg_plot.plot_1(y_test.flatten(), sample_rate=1000, title = 'Original')
#ecg_plot.plot_1(x_g_AB.flatten(), sample_rate=1000, title = 'Predicted')

In [ ]:
#x = y_test[0].flatten().tolist()
#y = x_g_AB[0].flatten().tolist()
#for i in range(len(x_g_AB)):
#  x.append((y_test[i][len(y_test[i])-1])[0])
#  y.append((x_g_AB[i][len(x_g_AB[i])-1])[0])

In [ ]:
print("acc: %3d%%"% accuracy_for_AtoB)

In [ ]:
# normalize correlation
a = (y_test - np.mean(y_test)) / (np.std(y_test) * len(y_test))
b = (x_g_AB - np.mean(x_g_AB)) / (np.std(x_g_AB))
c = np.correlate(a.flatten(), b.flatten(), 'full')
print(c)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test.flatten(), x_g_AB.flatten())
print(mse)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test.flatten(), x_g_AB.flatten())
print(mae)

In [ ]:
y_pred = x_g_AB.flatten()
y_test_new = y_test.flatten()

In [ ]:

y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0

In [ ]:

y_test_new[y_test_new > 0.5] = 1
y_test_new[y_test_new <= 0.5] = 0

In [ ]:
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score


In [ ]:

print("accuracy_score",accuracy_score(y_test_new, y_pred))

In [ ]:

print("precision_score",precision_score(y_test_new, y_pred , pos_label='positive', average='micro'))

In [ ]:

print("f1_score",f1_score(y_test_new, y_pred, pos_label='positive', average='micro'))

In [ ]:

print("recall_score",recall_score(y_test_new, y_pred, pos_label='positive', average='micro'))

In [ ]:
from sklearn.metrics import confusion_matrix
a = [1,1,1,1,1,0]
b=[1,1,1,1,0,0]
matrix = confusion_matrix(y_test_new, y_pred)
print(matrix)

In [ ]:
# correlation coefficient r of x and y
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test_new, y_pred)
print (r_value)

In [ ]:
# The coefficient of determination
from sklearn.metrics import r2_score
print(r2_score(y_test_new, y_pred))